# Tool selection example

When having too many tools, we can use a tool selection mechanism to choose the most appropriate tool for the task at hand.

## Manual approach

In [ ]:
import rich
from agentic_patterns.core.agents import get_agent, run_agent

### Create tools

In [ ]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


def sub(a: int, b: int) -> int:
    """Subtract two numbers"""
    print(f"Subtracting {a} - {b}")
    return a - b

In [ ]:
tools = [add, sub]
tools_by_name = {tool.__name__: tool for tool in tools}

In [ ]:
def func2descr(f):
    out = ""
    # Add the signature
    out += f"Tool: {f.__name__}({', '.join(f.__code__.co_varnames[: f.__code__.co_argcount])})\n"
    # Add the docstring
    if f.__doc__:
        out += f"Description: {f.__doc__}\n"
    # Add the return type
    if f.__annotations__ and "return" in f.__annotations__:
        out += f"Return type: {f.__annotations__['return']}\n"
    return out

In [ ]:
print(func2descr(add))

In [ ]:
tools_descriptions = [func2descr(tool) for tool in tools]
tools_descriptions_str = "\n".join(tools_descriptions)
print(tools_descriptions_str)

### Select tools

In [ ]:
agent = get_agent(output_type=list[str])

user_query = "What is the sum of 40123456789 and 2123456789?"

prompt = f"""
Given the user query, select the tools to use

User query: {user_query}

Tools available:
{tools_descriptions_str}

Answer only using the tool names
"""
result, _ = await run_agent(agent, prompt, verbose=True)
print(result.result.output)

In [ ]:
tool_names = result.result.output
print(f"Selected tools: {tool_names}")

### Use the tools

In [ ]:
# Only keep the tools that were selected
tools_agent = [tools_by_name[name] for name in tool_names if name in tools_by_name]  # type: ignore

# Create a new agent with the selected tools
agent = get_agent(tools=tools_agent)

prompt = user_query
result, nodes = await run_agent(agent, prompt, verbose=True)
print(result.result.output)

In [ ]:
rich.print(nodes)

## Using ToolSelector

In [ ]:
from agentic_patterns.core.tools import func_to_description, ToolSelector

# Show the improved function description
print(func_to_description(add))

In [ ]:
# Select tools
selector = ToolSelector([add, sub])
selected_tools = await selector.select(user_query, verbose=True)
selected_tools

In [ ]:
# Execute with selected tools
agent = get_agent(tools=selected_tools)
result, _ = await run_agent(agent, user_query, verbose=True)
print(result.result.output)